In [3]:
import pandas as pd
#import os
#pip install openpyxl
import numpy as np
import scipy as sp

In [61]:
Data=pd.read_excel("FiinPro_fiinpro_du-lieu-doanh-nghiep_08_12_2022181912753.xlsx",
                   header=7,
                   skipfooter=14
                  )


# LOADING Year: 2021
DataY21=Data.get(0)
DataY20=Data.get(1)
DataY19=Data.get(2)
DataY18=Data.get(3)




# RENAME COLUMNS
new_columns = {0: 'No', 
               1: 'Ma CK',
               2: 'Ten Cong ty',
               3: 'San',
               4: 'Nhom nganh',
               5: 'Tong Tai san',
               6: 'Tong No phai tra',
               7: 'Von Chu so huu',
               8: 'No ngan han',
               9: 'No dai han',
               10: 'Tai san co dinh',
               11: 'Gia von hang ban',
               12: 'Loi nhuan truoc thue',
               13: 'Chi phi lai vay',
               14: 'Von hoa thi truong',
               15: 'ROA'               
              }


#DataY21=DataY21.rename(columns={DataY21.columns[i]: new_col for i, new_col in new_columns.items()})

In [62]:
DataY21

In [79]:
Datas=pd.read_excel("Book1.xlsx",
                    sheet_name=[0,1,2],
                    header=5,
                   usecols="B:P",             skipfooter=15)
Data1=Datas.get(0)
# Data1 = Data1.drop([0,1])
Data1

,Mã CK,Tên,Sàn,Phân ngành - ICB Cấp: 1,A. TỔNG TÀI SẢN Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,I. NỢ PHẢI TRẢ Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,II. VỐN CHỦ SỞ HỮU Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,1. Nợ ngắn hạn Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,2. Nợ dài hạn Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,2. Tài sản cố định Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,4. Giá vốn hàng bán Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,16. Tổng lợi nhuận kế toán trước thuế Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,7.1. Trong đó: Chi phí lãi vay Hợp nhất Đã kiểm toán Năm: 2020 Quý: Hàng năm Đơn vị: VND,Vốn hóa thị trường (Marcap) Chỉ số TTM Ngày GD: 2022-08-11 Đơn vị: Tỷ VND,ROA % Chỉ số TTM Năm: 2022 Quý: 2 Đơn vị: %
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ticker,OrganShortName,ComGroupCode,DDE4,DDE16,DDE17,DDE18,DDE19,DDE20,DDE21,DDE22,DDE23,DDE24,DDE14,DDE15
2,CAR,Tập đoàn Giáo dục Trí Việt,UPCOM,Công nghiệp L1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.7488,0
3,GPC,Tập đoàn Green+,HNX,Dịch vụ Tiêu dùng L1,174785406644,24755996271,150029410373,9099596271,15656400000,18283392895,-12746853208,11760351754,-2077295896,0,NaN
4,PAT,Phốt pho Apatit Việt Nam,UPCOM,Nguyên vật liệu L1,855401502403,536126515223,319274987180,536126515223,0,438726581872,-1085407679101,84485321592,-12145855156,3334.45,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,XSQN,Xổ số Kiến thiết Quảng Ninh,OTC,Dịch vụ Tiêu dùng L1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,XTIPE,Công trình Thủy lợi Xuân Thủy,OTC,Công nghiệp L1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,YPHARCO,Dược phẩm Yên Bái,OTC,Dược phẩm và Y tế L1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2469,YRC,Đường sắt Yên Lào,OTC,Công nghiệp L1,84326541788,62780904996,21545636792,62780904996,0,17365250896,-114685549369,3060315508,-1631481371,NaN,NaN


In [18]:
DataY21=DataY21[~DataY21['Tong Tai san'].isna() 
                & DataY21['Von hoa thi truong']>0 ]
DataY21=DataY21[DataY21['Loi nhuan truoc thue']>0].sort_values(by=['Tong Tai san'], ascending=False)

In [21]:
DataY21.reset_index(drop=True, inplace=True)
DataY21

,No,Ma CK,Ten Cong ty,San,Nhom nganh,Tong Tai san,Tong No phai tra,Von Chu so huu,No ngan han,No dai han,Tai san co dinh,Gia von hang ban,Loi nhuan truoc thue,Chi phi lai vay,Von hoa thi truong,ROA
0,491,HPG,Hòa Phát,HOSE,Nguyên vật liệu L1,1.782364e+14,8.745580e+13,9.078063e+13,7.345932e+13,1.399648e+13,6.928084e+13,-1.085714e+14,3.705678e+13,-2.525823e+12,139264.117515,0.160962
1,671,MSN,Tập đoàn Masan,HOSE,Hàng Tiêu dùng L1,1.260935e+14,8.375682e+13,4.233665e+13,3.454784e+13,4.920898e+13,4.265394e+13,-6.649397e+13,1.148878e+13,-4.669426e+12,152623.296738,0.081446
2,1255,GVR,Tập đoàn CN Cao su VN,HOSE,Nguyên vật liệu L1,7.901420e+13,2.707416e+13,5.194004e+13,1.086685e+13,1.620731e+13,3.255896e+13,-1.861430e+13,6.212769e+12,-6.079996e+11,99400.000000,0.060679
3,129,BSR,Lọc - Hóa dầu Bình Sơn,UPCOM,Dầu khí L1,6.679566e+13,2.923186e+13,3.756380e+13,2.694044e+13,2.291413e+12,2.013609e+13,-9.338131e+13,6.940677e+12,-4.068114e+11,78954.222721,0.220559
4,749,PLX,Petrolimex,HOSE,Dầu khí L1,6.479124e+13,3.653105e+13,2.826019e+13,3.520709e+13,1.323963e+12,1.477879e+13,-1.563857e+14,3.789340e+12,-6.025274e+11,53301.344258,0.009902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,1179,VES,MÊ CA VNECO,UPCOM,Công nghiệp L1,1.814921e+10,5.207153e+09,1.294206e+10,5.207153e+09,0.000000e+00,0.000000e+00,-2.661526e+08,1.892145e+08,-2.406833e+08,36.930750,NaN
992,895,SCO,Công nghiệp Thủy Sản,UPCOM,Công nghiệp L1,1.483496e+10,1.101865e+11,-9.535154e+10,1.095188e+11,6.677000e+08,5.144267e+09,-8.913895e+09,1.555631e+09,-2.602210e+09,16.665270,NaN
993,247,AUM,Vinacafe Sơn Thành,UPCOM,Hàng Tiêu dùng L1,1.362615e+10,3.493688e+09,1.013246e+10,3.400688e+09,9.300002e+07,2.391953e+09,-1.050468e+10,4.357682e+07,0.000000e+00,10.500000,NaN
994,842,QNT,Tư vấn và Đầu tư Phát triển Quảng Nam,UPCOM,Công nghiệp L1,1.271185e+10,1.952708e+09,1.075914e+10,1.952708e+09,0.000000e+00,0.000000e+00,-1.002841e+09,1.085273e+09,0.000000e+00,16.110000,NaN


In [25]:
DataY21.describe()

,No,Tong Tai san,Tong No phai tra,Von Chu so huu,No ngan han,No dai han,Tai san co dinh,Gia von hang ban,Loi nhuan truoc thue,Chi phi lai vay,Von hoa thi truong,ROA
count,996.000000,9.960000e+02,9.960000e+02,9.960000e+02,9.960000e+02,9.960000e+02,9.960000e+02,9.960000e+02,9.960000e+02,9.960000e+02,996.000000,682.000000
mean,661.494980,2.695639e+12,1.470474e+12,1.225165e+12,1.053000e+12,4.174744e+11,7.189314e+11,-2.048736e+12,2.260157e+11,-3.997553e+10,2640.776473,0.064252
std,381.751445,9.807962e+12,5.420566e+12,4.754563e+12,3.843136e+12,2.301595e+12,3.409632e+12,8.438934e+12,1.424195e+12,1.955717e+11,13050.224598,0.073797
min,3.000000,1.164803e+10,1.257855e+09,-1.977530e+12,7.970584e+08,0.000000e+00,0.000000e+00,-1.563857e+14,1.390768e+07,-4.669426e+12,0.552000,-0.139254
25%,319.750000,1.771501e+11,5.664500e+10,8.981067e+10,4.906280e+10,6.305170e+07,1.841016e+10,-1.221666e+12,5.948531e+09,-1.980335e+10,78.999908,0.016990
50%,669.500000,5.335023e+11,2.430733e+11,2.324960e+11,1.907214e+11,6.207249e+09,6.947118e+10,-3.790807e+11,2.267527e+10,-3.688726e+09,243.125000,0.045211
75%,996.250000,1.503265e+12,7.780626e+11,6.612860e+11,6.433220e+11,9.105262e+10,3.056184e+11,-1.154589e+11,8.484834e+10,-1.676165e+08,917.150867,0.088462
max,1321.000000,1.782364e+14,8.745580e+13,9.078063e+13,7.345932e+13,4.920898e+13,6.928084e+13,0.000000e+00,3.705678e+13,1.862291e+11,186157.562634,0.545067


In [28]:
DataY21.groupby('Nhom nganh').mean()

,No,Tong Tai san,Tong No phai tra,Von Chu so huu,No ngan han,No dai han,Tai san co dinh,Gia von hang ban,Loi nhuan truoc thue,Chi phi lai vay,Von hoa thi truong,ROA
Nhom nganh,,,,,,,,,,,,
Công nghiệp L1,679.952083,1.932378e+12,1.121575e+12,8.108040e+11,6.973491e+11,4.242254e+11,5.557200e+11,-9.377279e+11,1.129843e+11,-3.038385e+10,1590.181209,0.053016
Công nghệ Thông tin L1,818.136364,3.453380e+12,2.015476e+12,1.437904e+12,1.724095e+12,2.913809e+11,5.503137e+11,-1.452600e+12,3.154628e+11,-3.059834e+10,4832.138793,0.072744
Dược phẩm và Y tế L1,573.346154,1.012631e+12,4.788105e+11,5.338205e+11,4.395837e+11,3.922679e+10,1.687965e+11,-8.234421e+11,7.574036e+10,-1.026933e+10,973.221548,0.077761
Dầu khí L1,658.818182,1.916249e+13,9.472047e+12,9.690443e+12,8.379899e+12,1.092147e+12,5.075562e+12,-2.982769e+13,1.175114e+12,-1.248079e+11,15719.687729,0.040595
Dịch vụ Tiêu dùng L1,629.623529,2.371384e+12,1.540852e+12,8.305321e+11,1.224120e+12,3.167323e+11,2.206906e+11,-2.532479e+12,1.404973e+11,-2.584198e+10,2853.105362,0.068877
Hàng Tiêu dùng L1,610.527174,3.240021e+12,1.625027e+12,1.614994e+12,1.175103e+12,4.499244e+11,8.144486e+11,-2.500476e+12,3.433060e+11,-5.479985e+10,4404.846958,0.074282
Nguyên vật liệu L1,688.623457,3.828177e+12,1.992766e+12,1.835412e+12,1.486567e+12,5.061989e+11,1.259119e+12,-3.161745e+12,4.442200e+11,-6.402214e+10,2988.206490,0.076381
